# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
import json
import subprocess
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


load_dotenv(override=True)

MODELS = {
    "GPT-4o-mini (OpenAI)": {
        "base_url": None,
        "api_key":  os.getenv("OPENAI_API_KEY"),
        "model":    "gpt-4o-mini",
        "tools":    True,   # tool-calling supported
    },
    "Gemini 2.0 Flash (Google)": {
        "base_url": "https://generativelanguage.googleapis.com/v1beta/openai/",
        "api_key":  os.getenv("GOOGLE_API_KEY"),
        "model":    "gemini-2.0-flash",
        "tools":    False,
    },
    "DeepSeek Chat": {
        "base_url": "https://api.deepseek.com/v1",
        "api_key":  os.getenv("DEEPSEEK_API_KEY"),
        "model":    "deepseek-chat",
        "tools":    False,
    },
    "Llama 3.2 (Local / Ollama)": {
        "base_url": "http://localhost:11434/v1",
        "api_key":  "ollama",
        "model":    "llama3.2",
        "tools":    False,
    },
}

def get_client(model_name: str) -> OpenAI:
    cfg = MODELS[model_name]
    if cfg["base_url"]:
        return OpenAI(base_url=cfg["base_url"], api_key=cfg["api_key"])
    return OpenAI(api_key=cfg["api_key"])

SYSTEM_PROMPT = """You are an expert programming tutor and AI/ML technical mentor.
You have deep knowledge of Python, software engineering, machine learning, LLMs, and system design.

When answering questions:
- Give clear, accurate explanations with code examples where relevant
- Break complex concepts into simple steps
- Highlight best practices and common pitfalls
- When it would help to *show* how code works, use the run_python_code tool to execute it live

You have access to a Python code execution tool — use it proactively to make your explanations concrete."""


run_code_tool = {
    "name": "run_python_code",
    "description": "Execute a Python code snippet and return its stdout/stderr. Use to demonstrate or verify how code works.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {"type": "string", "description": "The Python code to execute."}
        },
        "required": ["code"],
        "additionalProperties": False,
    },
}
tools = [{"type": "function", "function": run_code_tool}]


def run_python_code(code: str) -> str:
    """Execute code in a subprocess and return output."""
    try:
        result = subprocess.run(
            [sys.executable, "-c", code],
            capture_output=True, text=True, timeout=10,
        )
        out = result.stdout.strip()
        err = result.stderr.strip()
        if err:
            out += f"\n[stderr]: {err}"
        return out or "(no output)"
    except subprocess.TimeoutExpired:
        return "Error: timed out after 10 seconds"
    except Exception as e:
        return f"Error: {e}"


def handle_tool_calls(message):
    """Process tool calls, run them, and return API responses + display text."""
    api_responses = []
    display_parts = []
    for tc in message.tool_calls:
        if tc.function.name == "run_python_code":
            code = json.loads(tc.function.arguments).get("code", "")
            output = run_python_code(code)
            display_parts.append(
                f"**🔧 Code executed:**\n```python\n{code}\n```\n**Output:**\n```\n{output}\n```"
            )
            api_responses.append({
                "role": "tool",
                "content": output,
                "tool_call_id": tc.id,
            })
    return api_responses, display_parts



def chat(message: str, history: list, model_name: str):
    """
    Generator that yields progressively longer response strings.
    history is a list of [user_msg, bot_msg] tuples (Gradio default format).
    Handles tool calls (for models that support them) before streaming the reply.
    """
    client = get_client(model_name)
    cfg = MODELS[model_name]
    model = cfg["model"]

    msgs = [{"role": "system", "content": SYSTEM_PROMPT}]
    for h in history:
        msgs.append({"role": h["role"], "content": h["content"]})
    msgs.append({"role": "user", "content": message})

    prefix = ""  

  
    if cfg["tools"]:
        response = client.chat.completions.create(model=model, messages=msgs, tools=tools)
        while response.choices[0].finish_reason == "tool_calls":
            tool_msg = response.choices[0].message
            api_responses, display_parts = handle_tool_calls(tool_msg)
            prefix += "\n\n".join(display_parts) + "\n\n---\n\n"
            msgs.append(tool_msg)
            msgs.extend(api_responses)
            response = client.chat.completions.create(model=model, messages=msgs, tools=tools)


    try:
        stream = client.chat.completions.create(model=model, messages=msgs, stream=True)
        partial = prefix
        for chunk in stream:
            delta = chunk.choices[0].delta.content
            if delta:
                partial += delta
                yield partial
    except Exception as e:
        yield f"{prefix}⚠️ Error calling **{model_name}**: `{e}`"



with gr.Blocks(title="Tech Q&A Assistant", theme=gr.themes.Soft()) as ui:

    gr.Markdown(
        "# 🤖 Technical Q&A Assistant\n"
        "Ask any programming or AI/ML question. "
        "Switch models to compare answers. "
        "**GPT-4o-mini** can also *run Python code* live to illustrate explanations."
    )

    with gr.Row():
        model_dd = gr.Dropdown(
            choices=list(MODELS.keys()),
            value="GPT-4o-mini (OpenAI)",
            label="Model",
            scale=1,
            min_width=280,
        )
        gr.Markdown("*Tip: GPT-4o-mini supports live code execution via tool calling.*")

    chatbot = gr.Chatbot(height=520)

    with gr.Row():
        msg_box = gr.Textbox(
            placeholder="Ask a technical question…",
            label="",
            scale=5,
        )
        send_btn = gr.Button("Send ▶", variant="primary", scale=0, min_width=90)

    gr.Examples(
        examples=[
            ["Explain what this code does: yield from {book.get('author') for book in books if book.get('author')}"],
            ["What is the difference between a list and a generator in Python?"],
            ["Show me with a timing example how list comprehension compares to a for-loop"],
            ["Explain how the attention mechanism works in transformer models"],
            ["What are the trade-offs between RAG and fine-tuning an LLM?"],
        ],
        inputs=msg_box,
    )


    def user_turn(message, history):
        """Append user message dict and clear the input box."""
        return "", history + [{"role": "user", "content": message}]

    def bot_turn(history, model_name):
        """Stream the assistant reply, updating the last history entry."""
        last_user_msg = history[-1]["content"]
        prior_history = history[:-1]
        history_with_reply = history + [{"role": "assistant", "content": ""}]
        for partial in chat(last_user_msg, prior_history, model_name):
            history_with_reply[-1]["content"] = partial
            yield history_with_reply


    for trigger in (msg_box.submit, send_btn.click):
        trigger(user_turn, [msg_box, chatbot], [msg_box, chatbot]).then(
            bot_turn, [chatbot, model_dd], chatbot
        )

ui.launch(inbrowser=True)